In [ ]:
import pandas as pd
import altair as alt

cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## 6.3. Consultas Dinâmicas

_Consultas dinâmicas_ habilitam uma exploração reversível dos dados rápida para isolar padrões de interesse. Como definido por [Ahlberg, Williamson, &amp; Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), uma consulta dinâmica:

- representa uma consulta graficamente,
- fornece limites visuais para a faixa da consulta,
- fornece uma representação gráfica dos dados e o resultado da consulta,
- dá um retorno imediato do resultado após cada ajuste da consulta,
- e possibilita usuários novatos começar a trabalhar com pouco treino.

Uma abordagem comum é manipular os parâmetros da consulta usando ferramentas da interface do usuário padrão como uma barra deslizante, botões de rádio, e menus drop-down. Para gerar ferramentas de consulta dinâmica, podemos aplicar a operação `bind` da seleção para um ou mais campos de dados que queremos consultar.

Vamos construir um gráfico de dispersão interativo que usa uma consulta dinâmica para filtrar a exibição. Dado um gráfico de dispersão de avaliações de filmes (do Rotten Tomatoes e IMDB), podemos adicionar uma seleção sobre o campo `Major_Genre` para habilitar filtragens por gênero interativas.




Para começar, vamos extrair os gêneros (não nulos) dos dataset `movies`.

In [ ]:
df = pd.read_json(movies) # carregar dados de filmes
genres = df['Major_Genre'].unique() # obter valores de campo exclusivos
genres = list(filter(lambda d: d is not None, genres)) # filtrar valores
genres.sort() # classificar em ordem alfabética

Para uso futuro, vamos definir também uma lista de valores `MPAA_Rating` sem repetição:

In [ ]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora vamos criar uma seleção `single` ligada a um menu drop-down.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as avaliações variam por gênero? Como você revisaria o código para filtrar `MPAA_Rating` (G, PG, PG-13, etc.) ao invés de `Major_Genre`?*

In [ ]:
selectGenre = alt.selection_single(
    name='Select', # nomeie a seleção como 'Selection'
    fields=['Major_Genre'], # limitar a seleção ao campo Major_Genre
    init={'Major_Genre': genres[0]}, # use a primeira entrada do gênero como valor inicial
    bind=alt.binding_select(options=genres) # vincular-se a um menu de valores de gênero exclusivos
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

Nossa construção acima enaltece vários aspectos de seleções:

- Damos à seleção o nome (`'Select'`). Esse nome não é necessário, mas nos permite influenciar o rótulo de texto do menu de consulta dinâmica gerado. (_O que acontece se você remove o nome? Experimente!_)
- Restringimos a seleção a um campo de dado específico (`Major_Genre`). Anteriormente, quando usamos uma seleção `single`, a seleção era mapeada para pontos de dados individuais. Limitando a seleção para um campo específico podemos selecionar _todos_ os pontos do dado cujo valor do campo `Major_Genre` corresponde ao único valor selecionado.
- Inicializamos `init=...` a seleção para um valor inicial.
- `Vinculamos` a seleção para uma ferramenta da interface, nesse caso um menu drop-down via `binding_select`.
- Como antes, usamos uma codificação condicional para controlar o canal de opacidade.

### 6.3.1. Vincular Seleções para Várias Entradas

Uma instância de seleção pode ser vinculada a _várias_ ferramentas de consulta dinâmica. Vamos modificar o exemplo acima para fornecer filtros para _ambos_ `Major_Genre` e `MPAA_Rating`, usando botões de rádio ao invés de um menu. Nossa seleção `single` é definida agora sobre um único _par_ de gêneros e valores de classificação MPAA

_Procure por conjunções de gênero e classificação. Há algum filme de terror com classificação G ou PG?_

In [ ]:
# seleção de valor único em pares [Major_Genre, MPAA_Rating]
# use valores específicos conectados como os valores iniciais selecionados
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)
  
# gráfico de dispersão, modifique a opacidade com base na seleção
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

_Fatos interessantes: A classificação PG-13 não existia quando os filmes [Jaws](https://www.imdb.com/title/tt0073195/)  e [Jaws 2](https://www.imdb.com/title/tt0077766/) foram lançados. O primeiro filme a receber uma classificação PG-13 foi [Red Dawn](https://www.imdb.com/title/tt0087985/) de 1984._

### 6.3.2. Usando Visualizações como Consultas Dinâmicas

Embora ferramentas padrão de interface mostrem os valores dos parâmetros de consulta _possíveis_, elas não exibem a _distribuição_ desses valores. Podemos também querer usar interações mais ricas, como seleções `multi-value` e `interval`, ao invés de ferramentas de entrada que selecionam apenas um valor por vez.

Para resolver esses problemas, podemos criar gráficos adicionais tanto para visualizar dados e suportar consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e usar uma seleção `interval` para destacar dinamicamente filmes de períodos de tempo selecionados.

*Interaja com o histograma anual para explorar filmes de épocas diferentes. Viu alguma pista de [amostra enviesada](https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Como ano e avaliações da crítica se relacionam?)*

_Os anos vão de 1930 a 2040! Os filmes do futuro estão sendo produzidos ou há um erro de um século? Além disso, dependendo de qual fuso horário você está, talvez você veja uma pequena protuberância em 1969 ou 1970. Porque será? (Veja o final do notebook para uma explicação!)_

In [ ]:
brush = alt.selection_interval(
    encodings=['x'] # limitar a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmica
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# gráfico de dispersão, modifique a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

O exemplo acima fornece consultas dinâmicas usando uma _seleção interligada_ entre gráficos:

- Criamos uma seleção `interval` (`brush`) e colocamos `encodings=['x']` para limitar a seleção apenas para o eixo x, resultando numa seleção de intervalo unidimensional.
- Registramos `brush` com o nosso histograma de filme por ano via `.add_selection(brush)`.
- Usamos `brush` em uma codificação condicional para ajustar a `opacity` dos gráficos de dispersão.

Essa técnica de interação de selecionar elementos em um gráfico e observar destaques interligados em um ou mais gŕaficos é conhecida como [_varrer &amp; interligar_](https://en.wikipedia.org/wiki/Brushing_and_linking).